# XLSX creation FPI

### General Settings

In [1]:
import pandas as pd
import numpy as np
#import re as re
#import itertools as it
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
from pandas.io import sql
#from shutil import copyfile
#from subprocess import Popen
import winsound

In [2]:
#pd.set_option('display.max_columns', 1000)
#pd.set_option('display.max_rows', 10)

### Set parameter

In [3]:
p_countrycode='THA'
p_year=2016
p_mode='a' # r=replace, a = append

### Set directories

In [4]:
p_dir=r'\\TSHAMFIL901\iClick\Projects\Allianz\DB\Chartpack\XLSX'.replace(chr(92),'/')

In [5]:
p_subdir=p_dir +'/'+ r'data'.replace(chr(92),'/')
p_xlsx = 'FPI'

In [6]:
engine_dv_bpm = create_engine('mssql+pyodbc://TSMMHSQVS901/DV_BPM?driver=SQL+Server+Native+Client+11.0?trusted_connection=yes')

In [21]:
df=pd.read_sql_query(
    """SELECT *
    FROM DV_BPM.dbo.xls_FPI
    """, engine_dv_bpm)

In [22]:
dfg=df.groupby(['Country','LoBID','Year'],as_index=False)['SQR4'].mean()

In [23]:
df=pd.merge(left=df, right=dfg, left_on=('Country','LoBID','Year'), right_on=('Country','LoBID','Year'), how='left', suffixes=['','_mean'])

In [24]:
df['FPI']=(df.SQR4/df.SQR4_mean)*100
df_delta=df[['Country','LoBID','BrandID','Year','FPI','Q7_all_AwUnaided','Q8_AwAided']].copy()
df_delta['Year']=df_delta.Year+1
df=pd.merge(left=df, right=df_delta, left_on=('Country','LoBID','BrandID','Year'), right_on=('Country','LoBID','BrandID','Year'), how='left', suffixes=['','_PY'])
df['FPI_delta']=df.FPI-df.FPI_PY
df['Q7_all_AwUnaided_delta']=df.Q7_all_AwUnaided-df.Q7_all_AwUnaided_PY
df['Q8_AwAided_delta']=df.Q8_AwAided-df.Q8_AwAided_PY

In [25]:
df

Year              Country  LoBID  BrandID  \
0     2010               Turkey      2     5001   
1     2010  Australia Melbourne      2     5001   
2     2011               Turkey      1     5001   
3     2011               Poland      2     5001   
4     2011               Russia      2     5001   
5     2011              Ireland      2     5001   
6     2011     Australia Sydney      2     5001   
7     2012               Brazil      2     5001   
8     2012               Greece      2     5001   
9     2013              Croatia      2     5001   
10    2013               Turkey      2     5001   
11    2014              Croatia      1     5001   
12    2014               Turkey      1     5001   
13    2014             Bulgaria      2     5001   
14    2014              Ireland      2     5001   
15    2015             Bulgaria      1     5001   
16    2015             Portugal      2     5001   
17    2015               Brazil      2     5001   
18    2015               Greece      2     5001   
19    2016              Germany      2     5001   
20    2016              Croatia      2     5001   
21    2016               Turkey      2     5001   
22    2010              Hungary      1     5007   
23    2010              Romania      1     5013   
24    2010                China      1     5015   
25    2013                India      2     5016   
26    2016                India      2     5016   
27    2012                India      1     5017   
28    2015                India      1     5017   
29    2012            Australia      2     5019   
...    ...                  ...    ...      ...   
2022  2014             Bulgaria      1     5502   
2023  2013             Slovakia      1     5503   
2024  2012               Russia      2     5507   
2025  2011              Germany      1     5512   
2026  2013              Germany      2     5512   
2027  2014              Germany      1     5512   
2028  2016              Germany      2     5512   
2029  2011            Indonesia      1     5515   
2030  2013                India      2     5520   
2031  2016                India      2     5520   
2032  2012             Bulgaria      2     5524   
2033  2015             Bulgaria      2     5524   
2034  2012              Romania      2     5530   
2035  2015              Romania      2     5530   
2036  2016             Colombia      2     5542   
2037  2013                Italy      1     5551   
2038  2012              Ireland      2     5562   
2039  2012              Belgium      2     5572   
2040  2015                Korea      1     5574   
2041  2014              Croatia      2     5577   
2042  2015              Croatia      1     5577   
2043  2013              Romania      1     5579   
2044  2014             Malaysia      1     5584   
2045  2016             Portugal      2     5594   
2046  2014               Poland      1     5606   
2047  2015             Colombia      2     5611   
2048  2015                Italy      1     5641   
2049  2015              Hungary      1     5646   
2050  2016               Poland      1     5664   
2051  2016      Australia Perth      2     5710   

                                             Brand  Q7_all_AwUnaided  \
0                                          Allianz              27.0   
1                                          Allianz              36.6   
2                                          Allianz              22.8   
3                                          Allianz              41.1   
4                                          Allianz               9.0   
5                                          Allianz              33.8   
6                                          Allianz              37.8   
7                                          Allianz              16.1   
8                                          Allianz              13.2   
9                                          Allianz              58.2   
10                                         Allianz 

In [26]:
dfp=df[df.Year>p_year-3].pivot_table(index=['Country','LoBID','BrandID','Brand'], columns='Year',values='FPI', aggfunc='first').copy()
dfp.reset_index(inplace=True)

In [ ]:
#df

In [27]:
for country in df.Country.unique():
#    for lob in df.LoBID.unique():
    count11=len(df[((df.Country==country) & (df.Year==p_year)& (df.LoBID==1))])
    count12=len(dfp[((dfp.Country==country) &  (dfp.LoBID==1))])
    count21=len(df[((df.Country==country) & (df.Year==p_year)& (df.LoBID==2))])
    count22=len(dfp[((dfp.Country==country) &  (dfp.LoBID==2))])
#    print(p_dir+'/'+p_xlsx+'_'+country+'.xlsx')
    #LoBID 1:
    if count11+count21>0:
        writer = pd.ExcelWriter(p_dir+'/'+p_xlsx+'_'+country+'.xlsx', engine='xlsxwriter')
        workbook  = writer.book
        format = workbook.add_format({'font_color': 'orange','bg_color':'yellow'})
        df_row=4    
        df_col=1
        dfp_row=20
        dfp_col=1
        if count11>0:
            #df[((df.Country==country) & (df.Year==p_year)& (df.LoBID==1))].sort_values(['Country','LoBID','FPI'], ascending=[1,1,0]).to_excel(writer, sheet_name='FPI',index=False, startrow=4, startcol=1)
            #dfp[((dfp.Country==country)& (dfp.LoBID==1)) ].sort_values(['Country','LoBID',p_year], ascending=[1,1,0]).to_excel(writer, sheet_name='FPI',index=False, startrow=20, startcol=1)
            #worksheet0 = writer.sheets['FPI']
            #worksheet0.write_string(2,1, "FPI Life "+country)
            df[((df.Country==country) & (df.Year==p_year)& (df.LoBID==1))].sort_values(['Country','LoBID','FPI'], ascending=[1,1,0]).to_excel(writer, sheet_name='FPI_Life',index=False, startrow=df_row, startcol=df_col)
            dfp[((dfp.Country==country)& (dfp.LoBID==1)) ].sort_values(['Country','LoBID',p_year], ascending=[1,1,0]).to_excel(writer, sheet_name='FPI_Life',index=False, startrow=dfp_row, startcol=dfp_col)
            worksheet1 = writer.sheets['FPI_Life']
            worksheet1.write_string(df_row-1,df_col, "FPI_Life_0",format)
            worksheet1.write_string(df_row+count11+1,df_col, "FPI_Life_1",format)
            worksheet1.write_string(dfp_row-1,dfp_col, "FPI_Years_Life_0",format)
            worksheet1.write_string(dfp_row+count12+1,dfp_col, "FPI_Years_Life_1",format)
          #LoBID 2:
        if count21>0:
            #df[((df.Country==country) & (df.Year==p_year)& (df.LoBID==2))].sort_values(['Country','LoBID','FPI'], ascending=[1,1,0]).to_excel(writer, sheet_name='FPI',index=False, startrow=34, startcol=1)
            #dfp[((dfp.Country==country)& (dfp.LoBID==2)) ].sort_values(['Country','LoBID',p_year], ascending=[1,1,0]).to_excel(writer, sheet_name='FPI',index=False, startrow=50, startcol=1)
            #worksheet0 = writer.sheets['FPI']
            #worksheet0.write_string(32,1, "FPI P+C "+country)

            df[((df.Country==country) & (df.Year==p_year)& (df.LoBID==2))].sort_values(['Country','LoBID','FPI'], ascending=[1,1,0]).to_excel(writer, sheet_name='FPI_PuC',index=False, startrow=4, startcol=1)
            dfp[((dfp.Country==country)& (dfp.LoBID==2)) ].sort_values(['Country','LoBID',p_year], ascending=[1,1,0]).to_excel(writer, sheet_name='FPI_PuC',index=False, startrow=20, startcol=1)
            worksheet2 = writer.sheets['FPI_PuC']
            worksheet2.write_string(df_row-1,df_col, "FPI_PuC_0",format)
            worksheet2.write_string(df_row+count21+1,df_col, "FPI_PuC_1",format)
            worksheet2.write_string(dfp_row-1,dfp_col, "FPI_Years_PuC_0",format)
            worksheet2.write_string(dfp_row+count22+1,dfp_col, "FPI_Years_PuC_1",format)
        writer.save()

In [ ]:
import winsound
Freq = 3000 # Set Frequency To 3000 Hertz
Dur = 1000 # Set Duration To 1000 ms == 1 second
#winsound.Beep(Freq,Dur)